In [7]:
import os
import json
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.schema import Document

In [1]:
from langchain_community.retrievers import WikipediaRetriever

retriever = WikipediaRetriever()

In [4]:
docs = retriever.invoke("A.P.J.Abdul_Kalam")

In [5]:
docs

[Document(metadata={'title': 'A. P. J. Abdul Kalam', 'summary': 'Avul Pakir Jainulabdeen Abdul Kalam (  UB-duul kə-LAHM; 15 October 1931 – 27 July 2015) was an Indian aerospace scientist and statesman who served as the  president of India from 2002 to 2007.\nBorn and raised in a Muslim family in Rameswaram, Tamil Nadu, Kalam studied physics and aerospace engineering. He spent the next four decades as a scientist and science administrator, mainly at the Defence Research and Development Organisation (DRDO) and Indian Space Research Organisation (ISRO) and was intimately involved in India\'s civilian space programme and military missile development efforts. He was known as the "Missile Man of India" for his work on the development of ballistic missile and launch vehicle technology. He also played a pivotal organisational, technical, and political role in Pokhran-II nuclear tests in 1998, India\'s second such test after the first test in 1974.\nKalam was elected as the president of India i

In [8]:
os.environ["GOOGLE_API_KEY"] = "Insert API Key here"

In [9]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")


In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    """
    Answer the question based only on the context provided.
    Context: {context}
    Question: {question}
    """
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [26]:
chain.invoke(
    "when was he elected as president of india?"
)

'He was not elected as president of India. The context is about the vice president of India.'

In [33]:
format_docs(docs)

'Avul Pakir Jainulabdeen Abdul Kalam (  UB-duul kə-LAHM; 15 October 1931 – 27 July 2015) was an Indian aerospace scientist and statesman who served as the  president of India from 2002 to 2007.\nBorn and raised in a Muslim family in Rameswaram, Tamil Nadu, Kalam studied physics and aerospace engineering. He spent the next four decades as a scientist and science administrator, mainly at the Defence Research and Development Organisation (DRDO) and Indian Space Research Organisation (ISRO) and was intimately involved in India\'s civilian space programme and military missile development efforts. He was known as the "Missile Man of India" for his work on the development of ballistic missile and launch vehicle technology. He also played a pivotal organisational, technical, and political role in Pokhran-II nuclear tests in 1998, India\'s second such test after the first test in 1974.\nKalam was elected as the president of India in 2002 with the support of both the ruling Bharatiya Janata Part

In [30]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            formatted,
        ),
        ("human", "{input}"),
    ]
)

In [32]:
chain = prompt | llm
chain.invoke(
    {
        "input": "when was he elected as president of india?",
    })

AIMessage(content='A. P. J. Abdul Kalam was elected as the president of India in 2002.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--2d9c104f-1419-46fb-9772-f067d48a1b03-0', usage_metadata={'input_tokens': 2140, 'output_tokens': 23, 'total_tokens': 2163, 'input_token_details': {'cache_read': 0}})

In [34]:
chain.invoke(
    {
        "input": "where was he born?",
    })

AIMessage(content='Avul Pakir Jainulabdeen Abdul Kalam was born in the pilgrimage center of Rameswaram on Pamban Island, Madras Presidency (now in the Indian state of Tamil Nadu).', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--a8149e3f-bea1-4fa9-8114-e5fc3c73e93f-0', usage_metadata={'input_tokens': 2136, 'output_tokens': 40, 'total_tokens': 2176, 'input_token_details': {'cache_read': 0}})

In [35]:
chain.invoke(
    {
        "input": "list top 3 accomplishments of his career",
    })

AIMessage(content='Based on the provided text, here are three of A.P.J. Abdul Kalam\'s top accomplishments:\n\n1.  **Development of Ballistic Missile and Launch Vehicle Technology:** He was known as the "Missile Man of India" for his significant contributions to this area.\n2.  **Pivotal Role in Pokhran-II Nuclear Tests (1998):** He played a crucial organizational, technical, and political role in these tests, which were India\'s second set of nuclear tests.\n3.  **Presidency of India (2002-2007):** He served as the 11th president of India and was widely known as the "People\'s President."', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--1bfca360-c9cd-4d49-98c2-a7969a155a76-0', usage_metadata={'input_tokens': 2139, 'output_tokens': 151, 'total_tokens': 2290, 'input_token_details': {'cache_read': 0}})

In [36]:
chain.invoke(
    {
        "input": "explain like i'm 5 years old",
    })

AIMessage(content='Okay, imagine a really smart and kind grandpa named Abdul Kalam. He loved science and rockets!\n\nWhen he was a little boy, he had to sell newspapers to help his family. But he was also a very good student and loved learning.\n\nHe grew up to be a scientist who helped India build rockets and missiles. That\'s why people called him the "Missile Man of India"!\n\nLater, he became the President of India, which is like being the boss of the whole country! Everyone loved him because he was kind and always encouraged kids to study and learn. They called him the "People\'s President"!\n\nHe got lots of awards for being so smart and helpful. Even after he stopped being President, he kept teaching and writing books to inspire people.\n\nSadly, he died while giving a speech, but everyone remembers him as a great scientist, a kind leader, and someone who always wanted everyone to learn and do their best!\n\nSo, Abdul Kalam was a super smart and kind person who loved science and